In [7]:
import os
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
# now try to build **state** for Essay LLM-workflow

class EssayState(TypedDict):
  essay_text: str 
  cot_feedback: str 
  doa_feedback: str 
  lang_feedback: str
  summary_feedback:str 

  evaluation_score: Annotated[list[float]]
  final_score: float

In [ ]:
# globle llm model
llm= HuggingFaceEndpoint(
  repo_id="Qwen/Qwen2.5-1.5B-instruct",
  task="text-generation",
  huggingfacehub_api_token= os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

model= ChatHuggingFace(llm= llm)

## now try to build fn for COT(clearity of thought) node in graph

def cot(state:EssayState):

  class CotOutcome(BaseModel):
    cot_feedback: str= Field(description="Feedback about 'Clearity of Thought' on Essay")
    cot_score: float= Field(description="Evaluation score on Essay about 'Clearity of Thought' in out of 0 to 100")

  parser= PydanticOutputParser(pydantic_object=CotOutcome)

  prompt= PromptTemplate(
    template="Evaluate the Essay -> {essay_text} \n in the basis of 'Clearity of thought' and give output according to this format -> \n {format_instructions}",
    input_variables=['essay_text'],
    partial_variables={"format_instructions": parser.get_format_instructions()}
  )

  global model

  chain= prompt|model|parser 

  cot_feed_score= chain.invoke(state)

  return {"cot_feedback": cot_feed_score['cot_feedback'], "evaluation_score": cot_feed_score['cot_score']}



## now try to build a fn for DOA(depath of analysis) node

def doa(state:EssayState):

  class DoaOutcome(BaseModel):
    doa_feedback: str= Field(description="Feedback about 'Depath of Analysis' on Essay")
    doa_score: float= Field(description="Evaluate score on Essay about 'Depath of Analysis' in out of 0 to 100")

  parser= PydanticOutputParser(pydantic_object= DoaOutcome)

  prompt= PromptTemplate(
    template="Evaluate the essay -> {essay_text} \n on the basis of 'Depath of Analysis' and give the outcome according to this format -> \n {format_instructions}",
    input_variables=['essay_text'],
    partial_variables={"format_instructions":parser.get_format_instructions()}
  )

  global model

  chain= prompt|model|parser

  doa_feed_score= chain.invoke(state)

  return {"doa_feedback":doa_feed_score['doa_feedback'], "evaluation_score":doa_feed_score['doa_score']}

  